In [28]:
import sys
sys.path.append('..')
import gym

import torch
import torch.nn as nn


from models.td3 import TD3

In [29]:
class Actor(nn.Module):
    def __init__(self, nstates: int, nactions: int):
        super(Actor, self).__init__()
                
        self.fc1 = nn.Linear(nstates, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, nactions)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        
        return x

In [30]:
class Critic(nn.Module):
    def __init__(self, nstates: int, nactions: int):
        super(Critic, self).__init__()
                
        self.fc1 = nn.Linear(nstates + nactions, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, 1)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
                
    def forward(self, xs):
        x, a = xs
        
        x = self.relu1(self.fc1(torch.cat([x, a], 1)))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [31]:
lr = 1e-3
tau = 5e-3
eps = 0
gamma = 0.99

n_sts = 17
n_acts = 6

n_epis = 500
n_epoch = 200
n_rollout = 10

In [32]:
act = Actor(n_sts, n_acts)
act_opt = torch.optim.Adam(act.parameters(), lr=lr)

cri1 = Critic(n_sts, n_acts)
cri_opt1 = torch.optim.Adam(cri1.parameters(), lr=lr)

cri2 = Critic(n_sts, n_acts)
cri_opt2 = torch.optim.Adam(cri2.parameters(), lr=lr)

In [33]:
env = gym.make('HalfCheetah-v4', render_mode='human')

In [34]:
model = TD3(env, n_acts, act, act_opt, cri1, cri_opt1, cri2, cri_opt2, eps=eps, tau=tau, act_noise=0.2, act_range = (-1, 1))

In [35]:
model.train(n_epis, n_epoch, n_rollout)

step: 210, score: -5.393924261607379, n_buffer: 210
step: 410, score: -5.707643846801585, n_buffer: 410
step: 610, score: -5.563807125991809, n_buffer: 610
step: 810, score: -0.3876144188317863, n_buffer: 810
step: 1010, score: -7.181065132566765, n_buffer: 1010
step: 1210, score: -4.520551988461314, n_buffer: 1210
step: 1410, score: -4.33311632497443, n_buffer: 1410
step: 1610, score: -5.683141804594371, n_buffer: 1610
step: 1810, score: -5.640680781332599, n_buffer: 1810
step: 2210, score: -5.020859640526754, n_buffer: 2210
step: 2410, score: -0.3765556271406756, n_buffer: 2410
step: 2610, score: 0.023008534512446614, n_buffer: 2610
step: 2810, score: -2.051944812230227, n_buffer: 2810
step: 3010, score: -2.012657457757845, n_buffer: 3010
step: 3210, score: 2.5404797034422923, n_buffer: 3210
step: 3410, score: -0.7498075527480007, n_buffer: 3410
step: 3610, score: 0.9693244895748461, n_buffer: 3610
step: 3810, score: -0.11963938851873175, n_buffer: 3810
step: 4210, score: 0.060847308

KeyboardInterrupt: 

In [37]:
env.close()